In [79]:
#Data saved in csv/excel
import glob, os
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [80]:
#Set two decimal points for display
pd.options.display.float_format = "{:.4f}".format

In [81]:
df = pd.read_csv('C:\\Users\\tsb7592\\Downloads\\ForJulyWeighting\\NIHTBv3_demos.csv')
print(df.shape)

(3904, 77)


In [82]:
df.columns

Index(['FinalPIN', 'inPersonWillingLoc', 'region', '18yrsPlus', 'sex', 'age',
       'highestEd', 'hispanic', 'race_AmericanIndian', 'race_Asian',
       'race_Black', 'race_MiddleEast', 'race_Hawaiian',
       'race_OtherPacificIsl', 'race_White', 'race_unknwnNoAns', 'asian_Chin',
       'asian_Indian', 'asian_Filip', 'asian_Viet', 'asian_Korea',
       'asian_Japan', 'asian_OtherSpec', 'race_ethnicity', 'reg_Age',
       'reg_Education_l', 'reg_MothersEducation_l', 'reg_FathersEducation_l',
       'reg_GuardiansEducation', 'reg_StartingLevelOverride', 'reg_Gender_l',
       'reg_Handedness_l', 'reg_Race_l', 'reg_Ethnicity_l',
       'reg_multiple_data', 'reg_age_months', 'sex_l', '18yrsPlus_l',
       'highestEd_l', 'inPersonWillingLoc_l', 'hispanic_l',
       'race_AmericanIndian_l', 'race_Asian_l', 'race_Black_l',
       'race_MiddleEast_l', 'race_Hawaiian_l', 'race_OtherPacificIsl_l',
       'race_White_l', 'race_unknwnNoAns_l', 'asian_Chin_l', 'asian_Indian_l',
       'asian_Fili

# Recode Parental Education and Self Education

In [83]:
parental_edu={1:"less than high school graduate", 2:"high school graduate", 3:"some college",
4:"four year or higher degree", 5:'four year or higher degree' }

self_edu={1:"less than high school graduate", 2:"high school graduate", 3:"some college",
4:"Bachelors/4-year degree", 5:'Advanced Degree'}

In [84]:
df1=df[df.age< 18].replace({"highestEdu": parental_edu})

In [85]:
df1['highestEdu']

0                 high school graduate
1                         some college
2                         some college
3                         some college
4                 high school graduate
                     ...              
3897    less than high school graduate
3899    less than high school graduate
3900    less than high school graduate
3901    less than high school graduate
3902    less than high school graduate
Name: highestEdu, Length: 2248, dtype: object

In [86]:
#Recode Edu to norming value
df2= df[df.age>=18]['highestEdu'].map(self_edu)

In [87]:
edulabel =df1['highestEdu'].append(df2)

In [88]:
df=df.drop(columns=['highestEdu'])

In [89]:
df =pd.merge(df,edulabel,left_index=True, right_index=True)

In [90]:
df.tail(10)

,FinalPIN,inPersonWillingLoc,region,18yrsPlus,sex,age,highestEd,hispanic,race_AmericanIndian,race_Asian,...,race_sg_l,asian_type_sg,asian_type_sg_l,race,race_l,gender_l,ethnicity_l,highestEdu_l,NIHTB+MyCog,highestEdu
3894,WPX40108,12,4,1,2,49,3.0000,1.0000,0,0,...,White or Caucasian,NaN,NaN,1,White,Female,Not Hispanic,Some college,NaN,some college
3895,MSL09006,1,2,2,1,9,1.0000,1.0000,0,0,...,White or Caucasian,NaN,NaN,1,White,Male,Not Hispanic,Less than highschool,NaN,less than high school graduate
3896,MSL16003,1,2,2,2,16,1.0000,1.0000,0,0,...,White or Caucasian,NaN,NaN,1,White,Female,Not Hispanic,Less than highschool,NaN,less than high school graduate
3897,MSL08013,1,2,2,1,8,1.0000,1.0000,0,0,...,White or Caucasian,NaN,NaN,1,White,Male,Not Hispanic,Less than highschool,NaN,less than high school graduate
3898,MSL85001,1,2,1,2,90,4.0000,1.0000,0,0,...,White or Caucasian,NaN,NaN,1,White,Female,Not Hispanic,College,NaN,Bachelors/4-year degree
3899,MSL04001,1,2,2,1,4,1.0000,1.0000,0,0,...,White or Caucasian,NaN,NaN,1,White,Male,Not Hispanic,Less than highschool,NaN,less than high school graduate
3900,MSL11012,1,2,2,1,11,1.0000,1.0000,0,0,...,Black or African-American,NaN,NaN,2,Black,Male,Not Hispanic,Less than highschool,NaN,less than high school graduate
3901,MSL12001,1,2,2,2,12,1.0000,1.0000,0,0,...,Black or African-American,NaN,NaN,2,Black,Female,Not Hispanic,Less than highschool,NaN,less than high school graduate
3902,MSL09013,1,2,2,1,9,1.0000,1.0000,0,0,...,White or Caucasian,NaN,NaN,1,White,Male,Not Hispanic,Less than highschool,NaN,less than high school graduate
3903,MSL70106,1,2,1,1,74,4.0000,1.0000,0,0,...,White or Caucasian,NaN,NaN,1,White,Male,Not Hispanic,College,NaN,Bachelors/4-year degree


In [91]:
df.region.value_counts()

3    1436
4     940
2     866
1     662
Name: region, dtype: int64

In [92]:
keep =['FinalPIN','region', 'age','sex', 'highestEdu', 'hispanic', 'race_AmericanIndian', 'race_Asian',
       'race_Black', 'race_MiddleEast', 'race_Hawaiian',
       'race_OtherPacificIsl', 'race_White', 'race_unknwnNoAns']

In [93]:
df=df[keep].dropna()

# Label race/ethnicity

In [94]:
df.head()

,FinalPIN,region,age,sex,highestEdu,hispanic,race_AmericanIndian,race_Asian,race_Black,race_MiddleEast,race_Hawaiian,race_OtherPacificIsl,race_White,race_unknwnNoAns
0,MAP03001,2,3,2,high school graduate,1.0000,0,0,0,0,0,0,1,0
1,MAP03002,2,3,1,some college,1.0000,0,0,0,0,0,0,1,0
2,MAP03003,2,3,2,some college,1.0000,0,0,0,0,0,0,1,0
3,MAP03004,2,3,2,some college,1.0000,0,0,0,0,0,0,1,0
4,MAP03005,2,3,2,high school graduate,1.0000,0,0,0,0,0,0,1,0


In [95]:
df.hispanic.value_counts()

1.0000    3096
2.0000     484
5.0000     222
3.0000      81
4.0000      13
0.0000       6
Name: hispanic, dtype: int64

In [96]:
def label_race (row):
    #hispanic field does not equal 0 or 1
    if row['hispanic'] > 1 :
        return 'Hispanic'
    
    #hispanic=0 or 1, race_White=1 and/or race_MiddleEast=1, all other fields with prefix `race_` =0)
    elif row['race_White'] + row['race_MiddleEast'] >= 1 and row['race_AmericanIndian'] + row['race_Asian'] +row['race_Black'] + row['race_Hawaiian']+ row['race_OtherPacificIsl'] +row['race_unknwnNoAns']==0:
        return 'White'
    
    #race_Black=1, all other fields with prefix `race_` =0
    elif row['race_Black'] ==1  and row['race_White'] + row['race_MiddleEast'] + row['race_AmericanIndian'] + row['race_Asian'] + row['race_Hawaiian']+ row['race_OtherPacificIsl'] +row['race_unknwnNoAns']==0:
        return 'Black'
    
    #race_Asian=1, all other fields with prefix `race_` =0
    elif  row['race_Asian'] ==1  and row['race_White'] + row['race_MiddleEast'] + row['race_AmericanIndian'] + row['race_Black'] + row['race_Hawaiian']+ row['race_OtherPacificIsl'] +row['race_unknwnNoAns']==0:
        return 'AsianAmerican'
    
    else:
        return 'Other'    


In [97]:
df['Race/ethnicity'] =df.apply (lambda row: label_race(row), axis=1)

In [98]:
df['Race/ethnicity'].value_counts()

White            2162
Hispanic          800
Black             578
AsianAmerican     193
Other             169
Name: Race/ethnicity, dtype: int64

# Recode Region– Race/Ethnicity
there should be 20 race/ethnicity-by-region categories all together

In [99]:
df['region-race'] = df.region.map(str) + "-" + df['Race/ethnicity']

In [100]:
df['region-race'].unique()

array(['2-White', '2-Other', '2-Black', '2-Hispanic', '1-White',
       '2-AsianAmerican', '1-Black', '1-Hispanic', '1-Other',
       '1-AsianAmerican', '3-White', '3-Hispanic', '3-Black', '3-Other',
       '3-AsianAmerican', '4-Hispanic', '4-Black', '4-Other', '4-White',
       '4-AsianAmerican'], dtype=object)

In [101]:
df.head()

,FinalPIN,region,age,sex,highestEdu,hispanic,race_AmericanIndian,race_Asian,race_Black,race_MiddleEast,race_Hawaiian,race_OtherPacificIsl,race_White,race_unknwnNoAns,Race/ethnicity,region-race
0,MAP03001,2,3,2,high school graduate,1.0000,0,0,0,0,0,0,1,0,White,2-White
1,MAP03002,2,3,1,some college,1.0000,0,0,0,0,0,0,1,0,White,2-White
2,MAP03003,2,3,2,some college,1.0000,0,0,0,0,0,0,1,0,White,2-White
3,MAP03004,2,3,2,some college,1.0000,0,0,0,0,0,0,1,0,White,2-White
4,MAP03005,2,3,2,high school graduate,1.0000,0,0,0,0,0,0,1,0,White,2-White


In [102]:
def crosstab(df):
    a =df.groupby(['region','Race/ethnicity'])['age'].count().reset_index()
    a.columns=['region','Race/ethnicity','count']
    aa =a.pivot_table(values=['count'], index=['region'], columns=['Race/ethnicity'])
    aa=aa.fillna(0)
    aa.loc['Total',:]= aa.sum(axis=0)
    aa["sum"] = aa.sum(axis=1)
    
    return aa

In [103]:
dfr =crosstab(df)
dfr

count                                            sum
Race/ethnicity AsianAmerican    Black Hispanic    Other     White          
region                                                                     
1                    37.0000  89.0000 103.0000  21.0000  411.0000  661.0000
2                    21.0000 109.0000  76.0000  31.0000  628.0000  865.0000
3                    58.0000 329.0000 272.0000  67.0000  710.0000 1436.0000
4                    77.0000  51.0000 349.0000  50.0000  413.0000  940.0000
Total               193.0000 578.0000 800.0000 169.0000 2162.0000 3902.0000

# Recode Sex assigned at birth
there should be 8 sex-by-region categories all together

In [104]:
df["sex"].replace({1:'Male', 2:'Female'}, inplace=True)

In [111]:
df['region-sex'] = df.region.map(str) + "-" + df['sex']

In [112]:
df['region-sex'].unique()

array(['2-Female', '2-Male', '1-Female', '1-Male', '3-Male', '3-Female',
       '4-Male', '4-Female'], dtype=object)

# Split Age into 4 subests:ages 3-17, ages 18-24, ages 25-59, and ages 60+

In [107]:
bins= [3,18,25,60, 200]
labels = ['Child','YoungAdult','MiddleAdult','OlderAdult']
df['agegroup'] = pd.cut(df['age'], bins=bins, labels=labels, right=False)

In [108]:
Child= df[df['agegroup']=='Child']
Child.highestEdu.value_counts()

four year or higher degree        907
some college                      721
high school graduate              560
less than high school graduate     58
Name: highestEdu, dtype: int64

In [ ]:
'''
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('C:/Users/tsb7592/Downloads/NIHTB_counts_July.xlsx', engine='xlsxwriter')

# Write each dataframe to a different worksheet.
aa.to_excel(writer, sheet_name='adults')
kk.to_excel(writer, sheet_name='kids')  
ad['highestEd'].value_counts().to_excel(writer, sheet_name='ad_edu') 
kd['highestEd'].value_counts().to_excel(writer, sheet_name='kids_edu') 
# Close the Pandas Excel writer and output the Excel file.
writer.save()
'''

# Save for R code

In [113]:
final =['FinalPIN','region', 'age', 'sex', 'highestEdu','region-race','region-sex','agegroup']

In [114]:
df[final].tail()

,FinalPIN,region,age,sex,highestEdu,region-race,region-sex,agegroup
3899,MSL04001,2,4,Male,less than high school graduate,2-White,2-Male,Child
3900,MSL11012,2,11,Male,less than high school graduate,2-Black,2-Male,Child
3901,MSL12001,2,12,Female,less than high school graduate,2-Black,2-Female,Child
3902,MSL09013,2,9,Male,less than high school graduate,2-White,2-Male,Child
3903,MSL70106,2,74,Male,Bachelors/4-year degree,2-White,2-Male,OlderAdult


In [115]:
df[final].to_csv('C:/Users/tsb7592/Downloads/NIHTB_demoFinal_July.csv', index=False)

# Validate (After running the Rcode)

In [169]:
pedu={'less_HS_prop':"less than high school graduate", 'HS_GED_prop':"high school graduate", 'SomeCollege_prop':"some college",
'BA_more_prop':'four year or higher degree' }

sedu={'less_HS_prop':"less than high school graduate", 'HS_GED_prop':"high school graduate", 'SomeCollege_prop':"some college",
'Bachelors_prop':"Bachelors/4-year degree", 'Advanced_prop':'Advanced Degree'}

In [166]:
#Read the output files from R code
child = pd.read_csv('C:/Users/tsb7592/Downloads/child_weights.csv')
yadult = pd.read_csv('C:/Users/tsb7592/Downloads/youngadults_weights.csv')

In [172]:
tar = pd.read_csv('C:\\Users\\tsb7592\\Downloads\\ForJulyWeighting\\ChildTargets.csv')
ytar = pd.read_csv('C:\\Users\\tsb7592\\Downloads\\ForJulyWeighting\\YoungAdultTargets.csv')

In [162]:
import numpy as np

def obtained_weighted(data, target, edu):
    '''
    Fill the obtained N for race, sex and edu
    Use the raw counts to fill in the “unweighted_N” 
    
    '''
    target = target.replace({"Target": edu})
    target['name'] = target.REGION.map(str) + "-" + target['Target']
    rr = data.groupby('region.race')['FinalPIN'].count().reset_index()
    rs = data.groupby('region.sex')['FinalPIN'].count().reset_index()
    re = data.groupby('highestEdu')['FinalPIN'].count().reset_index()
    
    comb = pd.merge(target, rr, left_on='name', right_on='region.race', how='left')
    comb['Obtained_N'].fillna(comb['FinalPIN'],inplace=True)
    
    comb = pd.merge(comb, rs, left_on='name', right_on='region.sex', how='left')
    comb['Obtained_N'].fillna(comb['FinalPIN_y'],inplace=True)
    
    comb = pd.merge(comb, re, left_on='Target', right_on='highestEdu', how='left')
    comb['Obtained_N'].fillna(comb['FinalPIN'],inplace=True)
    
    #“unweighted_proportion” is the unweighted_N divided by the sum of the unweighted_n grouped-by characteristic. 
    rr = data.groupby('region.race')['weights4'].sum().reset_index()
    rs = data.groupby('region.sex')['weights4'].sum().reset_index()
    re = data.groupby('highestEdu')['weights4'].sum().reset_index()
    
    comb = pd.merge(comb, rr, left_on='name', right_on='region.race', how='left')
    comb['Weighted_Proportion'].fillna(comb['weights4'],inplace=True)
    
    comb = pd.merge(comb, rs, left_on='name', right_on='region.sex', how='left')
    comb['Weighted_Proportion'].fillna(comb['weights4_y'],inplace=True)
    
    comb = pd.merge(comb, re, left_on='Target', right_on='highestEdu', how='left')
    comb['Weighted_Proportion'].fillna(comb['weights4'],inplace=True)    
    
    return comb.iloc[:,:10]

In [170]:
childtar =obtained_weighted(child, tar, pedu)
childtar.head()

,ageCat,REGION,Characteristic,Target,Proportion,Obtained_N,Unweighted_Proportion,Weighted_Proportion,Unweighted_Difference,Weighted_Difference
0,1,1,raceEth,AsianAmerican,0.0114,19.0000,NaN,25.6070,NaN,NaN
1,1,1,raceEth,Black,0.0199,52.0000,NaN,44.6999,NaN,NaN
2,1,1,raceEth,Hispanic,0.0333,69.0000,NaN,74.7993,NaN,NaN
3,1,1,raceEth,Other,0.0066,20.0000,NaN,14.8251,NaN,NaN
4,1,1,raceEth,White,0.0893,188.0000,NaN,200.5879,NaN,NaN


In [ ]:
def crosstab(df, col='weights'):
    a =(df.groupby(['region','race'])[col].count()/df[col].count()).reset_index()
    a.columns=['region','race','Sum']
    aa =a.pivot_table(values=['Sum'], index=['region'], columns=['race'])
    return aa

In [ ]:
kp = crosstab(kids)
kp

In [ ]:
ap = crosstab(adults,'weights.7a')
ap

In [ ]:
def crosstab_w(df, col='weights'):
    a =(df.groupby(['region','race'])[col].sum()/df[col].sum()).reset_index()
    a.columns=['region','race','Sum']
    aa =a.pivot_table(values=['Sum'], index=['region'], columns=['race'])
    return aa

In [ ]:
k7 = crosstab_w(kids)
k7

In [ ]:
a7=crosstab_w(adults, 'weights.7a')
a7

In [ ]:
b7 =(adults.groupby(['region'])['weights.7b'].count()/adults['weights.7b'].sum()).reset_index()
b7.columns=['region','Sum']
b77 =b7.pivot_table(values=['Sum'], index=['region'])
b77

In [ ]:
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('C:/Users/tsb7592/Downloads/NIHTB_weights_output_April.xlsx', engine='xlsxwriter')

# Write each dataframe to a different worksheet.
kp.to_excel(writer, sheet_name='kids_unweighted')
ap.to_excel(writer, sheet_name='adults_unweighted')
k7.to_excel(writer, sheet_name='kids_obtained_weighted')
a7.to_excel(writer, sheet_name='adults_obtained_weighted_7a') 
b77.to_excel(writer, sheet_name='adults_obtained_weighted_7b') 

# Close the Pandas Excel writer and output the Excel file.
writer.save()